In [ ]:
import pickle
from google.colab import drive
import os
import pprint
import pandas as pd
from tensorflow.keras.utils import plot_model
from keras.models import load_model,Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
from keras.utils import np_utils

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.65.95.82:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5685545286329333921),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -5398041485626348402),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7715833888834233905),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, -1235918857523894948),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, -4013830805176225422),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -2237730334099227710),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 1640355365131669915),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, -2107329395461208926),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184,

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.65.95.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.65.95.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with open("/content/drive/MyDrive/FT_300/sentiment/sentiment_params.pickle",'rb') as file:
  ft_file = pickle.load(file)

with open("/content/drive/MyDrive/GLOVE_100/sentiment/sentiment_params.pickle",'rb') as file:
  gl100_file = pickle.load(file)

with open("/content/drive/MyDrive/GLOVE_200/sentiment/sentiment_params.pickle",'rb') as file:
  gl200_file = pickle.load(file)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/tr_sen_fin.csv')

In [ ]:
def load_all_models(names):
  with tf.device('/job:localhost'):
      # for name in names:
      # # define filename for this ensemble
      #   filename = '/content/drive/MyDrive/sentiment models/'+name
      # # load model from file
      # models = [load_model('/content/drive/MyDrive/sarcasm models/'+name) for name in names]
    models = [load_model('/content/drive/MyDrive/'+name) for name in names]
      # add to list of members
        # all_models.append(model)
        # print('>loaded %s' % name)
  return models
  

In [ ]:
# from keras.utils import plot_model
def define_stacked_model(names,members):
	# update all layers in all models to not be trainable
  for i in range(len(members)):
    model = members[i]
    # model.input._name = names[i].replace('.h5','')+'_'+model.input.name
    for layer in model.layers:
      # make not trainable
      layer.trainable = False
      # rename to avoid 'unique layer name' issue
      layer._name = names[i].replace('.h5','')+'_' + 'ensemble'+'_'+str(i+1) + '_' + layer.name
    # define multi-headed input
  ensemble_visible = [model.input for model in members]
  # concatenate merge output from each model
  ensemble_outputs = [model.output for model in members]
  merge = concatenate(ensemble_outputs)
  hidden = Dense(16, activation='relu')(merge)
  output = Dense(5, activation='softmax')(hidden)
  model = Model(inputs=ensemble_visible, outputs=output)
  # plot graph of ensemble
  # plot_model(model, show_shapes=True, to_file='model_graph.png')
  # compile
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [ ]:
def folds(x):
  df_tr = df[df['kfold']!=x].reset_index(drop=True)
  df_val = df[df['kfold']==x].reset_index(drop=True)
  tr_text = df_tr['text'].values
  val_text = df_val['text'].values  
  x_tr_gl_100 = tf.convert_to_tensor(pad_sequences(gl100_file['Tokenizer'].texts_to_sequences(tr_text),gl100_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_tr_gl_200 = tf.convert_to_tensor(pad_sequences(gl200_file['Tokenizer'].texts_to_sequences(tr_text),gl200_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_tr_ft_300 = tf.convert_to_tensor(pad_sequences(ft_file['Tokenizer'].texts_to_sequences(tr_text),ft_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)


  # val_text = df_val['text'].values
  x_val_gl_100 = tf.convert_to_tensor(pad_sequences(gl100_file['Tokenizer'].texts_to_sequences(val_text),gl100_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_val_gl_200 = tf.convert_to_tensor(pad_sequences(gl200_file['Tokenizer'].texts_to_sequences(val_text),gl200_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)
  x_val_ft_300 = tf.convert_to_tensor(pad_sequences(ft_file['Tokenizer'].texts_to_sequences(val_text),ft_file['long_sen_num'], padding='post',truncating = 'post'),np.int32)

  y_val = np_utils.to_categorical(df_val.lbl_num.values)
  y_tr = np_utils.to_categorical(df_tr.lbl_num.values)

  y_val = tf.convert_to_tensor(y_val,np.float32)
  y_tr = tf.convert_to_tensor(y_tr,np.float32)

  return {'x_tr_gl_100':x_tr_gl_100,'x_tr_gl_200':x_tr_gl_200,'x_tr_ft_300':x_tr_ft_300,'y_tr':y_tr,'x_val_gl_100':x_val_gl_100,'x_val_gl_200':x_val_gl_200,'x_val_ft_300':x_val_ft_300,'y_val':y_val}

In [ ]:
initial_learning_rate = 0.01
epochs = 100
decay = initial_learning_rate / epochs
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

In [ ]:

# stacked.fit(x = [x_tr_gl_200,x_tr_gl_100,x_tr_ft_300],y = y_tr,validation_data=([x_val_gl_200,x_val_gl_100,x_val_ft_300],y_val),epochs=100,callbacks=[EarlyStopping()])
def train_stacked():
  # stacked1 = ['FT_300/sentiment/sentiment_SEP_CONV1D.h5','FT_300/sentiment/sentiment_BiGRU.h5','GLOVE_100/sentiment/sentiment_CONV1D.h5','GLOVE_200/sentiment/sentiment_CONV1D_BiLSTM.h5']
  # stacked2 = ['FT_300/sentiment/sentiment_BiLSTM.h5','FT_300/sentiment/sentiment_CONV1D.h5','GLOVE_200/sentiment/sentiment_BiRNN.h5','GLOVE_200/sentiment/sentiment_SEP_CONV1D_BiGRU.h5']
  stacked3 = ['FT_300/sentiment/sentiment_CONV1D_BiLSTM.h5','FT_300/sentiment/sentiment_BiGRU.h5','GLOVE_100/sentiment/sentiment_CONV1D_BiRNN.h5','GLOVE_100/sentiment/sentiment_BiRNN.h5']
  stacked4 = ['FT_300/sentiment/sentiment_BiLSTM.h5','FT_300/sentiment/sentiment_CONV1D.h5','GLOVE_100/sentiment/sentiment_BiRNN.h5','GLOVE_100/sentiment/sentiment_CONV1D_BiGRU.h5']
  # stacked5 = ['FT_300/sentiment/sentiment_CONV1D.h5','GLOVE_100/sentiment/sentiment_BiGRU.h5','GLOVE_200/sentiment/sentiment_CONV1D_BiLSTM.h5','GLOVE_200/sentiment/sentiment_BiLSTM.h5']
  # stacked6 = ['FT_300/sentiment/sentiment_CONV1D.h5','FT_300/sentiment/sentiment_BiLSTM.h5','FT_300/sentiment/sentiment_CONV1D_BiLSTM.h5','FT_300/sentiment/sentiment_SEP_CONV1D.h5','FT_300/sentiment/sentiment_BiGRU.h5']
  # stacked7 = ['GLOVE_100/sentiment/sentiment_CONV1D.h5','GLOVE_100/sentiment/sentiment_CONV1D_BiRNN.h5','GLOVE_100/sentiment/sentiment_BiRNN.h5','GLOVE_100/sentiment/sentiment_CONV1D_BiGRU.h5','GLOVE_100/sentiment/sentiment_BiGRU.h5']
  # stacked8 = ['GLOVE_200/sentiment/sentiment_CONV1D_BiLSTM.h5','GLOVE_200/sentiment/sentiment_BiRNN.h5','GLOVE_200/sentiment/sentiment_SEP_CONV1D_BiGRU.h5','GLOVE_200/sentiment/sentiment_BiLSTM.h5']

  model_config = {}
  # model_config['STACKED_1'] = (stacked1,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_100','x_tr_gl_200'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_200']) #300 300 100 200
  # model_config['STACKED_2'] = (stacked2,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_200','x_tr_gl_200'],['x_val_ft_300','x_val_ft_300','x_val_gl_200','x_val_gl_200']) #300 300 200 200
  model_config['STACKED_3'] = (stacked3,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_100','x_tr_gl_100'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_100']) #300 300 100 100
  model_config['STACKED_4'] = (stacked4,['x_tr_ft_300','x_tr_ft_300','x_tr_gl_100','x_tr_gl_100'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_100']) #300 300 100 100
  # model_config['STACKED_5'] = (stacked5,['x_tr_ft_300','x_tr_gl_100','x_tr_gl_200','x_tr_gl_200'],['x_val_ft_300','x_val_ft_300','x_val_gl_100','x_val_gl_100']) #300 100 200 200
  # model_config['STACKED_6'] = (stacked6,['x_tr_ft_300','x_tr_ft_300','x_tr_ft_300','x_tr_ft_300','x_tr_ft_300'],['x_val_ft_300','x_val_ft_300','x_val_ft_300','x_val_ft_300','x_val_ft_300']) #300 300 300 300 300
  # model_config['STACKED_7'] = (stacked7,['x_tr_gl_100','x_tr_gl_100','x_tr_gl_100','x_tr_gl_100','x_tr_gl_100'],['x_val_gl_100','x_val_gl_100','x_val_gl_100','x_val_gl_100','x_val_gl_100']) #100 100 100 100 100
  # model_config['STACKED_8'] = (stacked8,['x_tr_gl_200','x_tr_gl_200','x_tr_gl_200','x_tr_gl_200'],['x_val_gl_200','x_val_gl_200','x_val_gl_200','x_val_gl_200']) #200 200 200 200 

  for k,v in model_config.items():
    with strategy.scope():
      members = load_all_models(v[0])
      stacked = define_stacked_model(v[0],members)
    
    plot_model(stacked, show_shapes=True, to_file='/content/drive/MyDrive/STACKED/sentiment/sentiment_'+k+'.png')
    print("----------------------::  TRAINING STARTED :- "+k+" ::----------------------\n")
    for i in range(5):
      print("-----------:: FOLD :- "+str(i+1)+' ::-----------')
      data = folds(i)
      x_t = [data[x] for x in v[1]]
      y_t = data['y_tr']
      x_v = [data[x] for x in v[2]]
      y_v = data['y_val']
      stacked.fit(x = x_t,y = y_t,validation_data=(x_v,y_v),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=2),ModelCheckpoint(filepath='/content/drive/MyDrive/STACKED/sentiment/sentiment_'+k+'.h5', monitor='val_loss', save_best_only=True),LearningRateScheduler(lr_time_based_decay,verbose=1)])
    print("----------------------::  TRAINING COMPLETED :- "+k+" ::----------------------\n")

In [ ]:
train_stacked()

----------------------::  TRAINING STARTED :- STACKED_3 ::----------------------

-----------:: FOLD :- 1 ::-----------
Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
17250/17250 [==============================] - 563s 30ms/step - loss: 0.5216 - accuracy: 0.8255 - val_loss: 0.4977 - val_accuracy: 0.8274
Epoch 2/100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0009999000574917021.
17250/17250 [==============================] - 504s 29ms/step - loss: 0.4525 - accuracy: 0.8442 - val_loss: 0.4936 - val_accuracy: 0.8272
Epoch 3/100

Epoch 00003: LearningRateScheduler reducing learning rate to 0.000999700106714659.
17250/17250 [==============================] - 504s 29ms/step - loss: 0.4504 - accuracy: 0.8434 - val_loss: 0.4903 - val_accuracy: 0.8261
Epoch 4/100

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0009994003415259673.
17250/17250 [==============================] - 502s 29ms/step - loss: 0.4441 - 